In [4]:
#!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.0 MB 3.3 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.0 MB 3.4 MB/s eta 0:00:03
   --------- ------------------------------ 2.1/9.0 MB 3.6 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.0 MB 3.8 MB/s eta 0:00:02
   ----------------- ---------------------- 3.9/9.0 MB 4.0 MB/s eta 0:00:02
   ---------------------- ----------------- 5.0/9.0 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 6.0/9.0 MB 4.3 MB/s eta 0:00:01
   -------------------------------- ------- 7.3/9.0 MB 4.5 MB/s eta 0:00:01
   -------------------------------------- - 8.7/9.0 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 4.7 MB/s  0:00:01
   ---------------------------------------- 0.0/795.4 kB ? eta -:--:--
   ----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\inter\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import joblib
import streamlit as st
import pandas as pd
import numpy as np
from Classes import Categorical_Correction, Outlier_Treatment, Missing_Value_Treatment, Typecasting, Feature_Engineering, Encoder, Scaler, Feature_Selection

random_state = 42

In [6]:
# Carregar modelo
model = joblib.load("modelo_mlp_teste.pkl")

In [7]:
st.title("Get the BEST PRICE for your car")

method = st.sidebar.radio("How to insert the car information",
                          ["Load a csv file", "Manually write the information"])


if method == "Load a csv file":

    st.subheader("Load a csv file")
    file = st.file_uploader("Upload a csv file", type="csv")
    if file is not None:
        df = pd.read_csv(file)

        necessary_columns = set(model.feature_names_in_) # original columns, before feature selection
        received_columns = set(df.columns)

        # make sure that the file uploaded has the columns needed for the model
        if not necessary_columns.issubset(received_columns):
            st.error("The file does not contain the necessary columns")
            st.write("Missing columns:" + str(necessary_columns - received_columns))
            st.stop()

        # in case the file has the needed columns and more, we only keep the necessary columns
        df = df[list(necessary_columns)]

        st.write("Data loaded:")
        st.dataframe(df)

        if st.button("Predict Price"):
            try:
                df.set_index('carID', inplace=True)

                # transform inconsistencies in NA
                df.loc[df['year']>2020, 'year'] = np.nan
                df.loc[df['mileage']<0, 'mileage'] = np.nan
                df.loc[df['tax']<0, 'tax'] = np.nan
                df.loc[df['mpg']<=0, 'mpg'] = np.nan
                df.loc[df['previousOwners']< 0, 'previousOwners'] = np.nan
                df.loc[df['engineSize']<= 0, 'engineSize'] = np.nan
                df.loc[df['mpg'] < 40, 'mpg'] = np.nan
                df.loc[df['engineSize'] < 1, 'engineSize'] = np.nan

                # round features
                df['year'] = np.floor(df['year'])
                df['previousOwners'] = np.floor(df['previousOwners'])

                for feat in ['mileage', 'tax', 'mpg', 'engineSize']:
                    df[feat] = df[feat].round(2)


                # pre-processing of the categorical features (capitalize and remove spaces)

                df['Brand'] = df['Brand'].where(df['Brand'].isna(), df['Brand'].astype(str).str.strip().str.upper())

                df['model'] = df['model'].where(df['model'].isna(), df['model'].astype(str).str.strip().str.upper())

                df['fuelType'] = df['fuelType'].where(df['fuelType'].isna(), df['fuelType'].astype(str).str.strip().str.upper())

                df['transmission'] = df['transmission'].where(df['transmission'].isna(), df['transmission'].astype(str).str.strip().str.upper())

                # drop paintQuality% since that is evaluated by the mechanic
                df.drop('paintQuality%', axis=1, inplace=True)


                # predict
                pred = model.predict(df)
                st.write("Predictions:")
                df_results = df.copy()
                df_results['pred'] = pred
                st.dataframe(df_results)

                # possible to download results
                st.download_button("Download Predictions", df_results.to_csv("predictions.csv", index=False))

            except Exception as e:
                st.error("An error occurred: " + str(e))


if method == "Manually write the information":
    st.subheader("Manually write the information")

    # empty, single-lined data frame with columns with the names of the excepted features
    observation = pd.Dataframe(columns = model.feature_names_in_)

    feature_values = {}
    st.write("Insert the values for each feature")

    for feat in observation.columns:
        value = st.text_input(feat)
        if value.strip() == "":
            value = np.nan
        feature_values[feat] = value


    if str.button("Predict Price"):
        try:
            df_observation = pd.DataFrame([feature_values])

            # convert the numeric values from string to numeric
            df_observation = df_observation.apply(pd.to_numeric, errors='ignore')

            # predict price
            prediction = model.predict(df_observation)[0] # with [0] we select only the value, otherwise it would be an np.array just with the value

            st.success("Predicted price:" + str(prediction))

        except Exception as e:
            st.error("An error occurred: " + str(e))







2025-12-07 18:37:24.651 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 18:37:24.902 
  command:

    streamlit run C:\Users\inter\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-07 18:37:24.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 18:37:24.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 18:37:24.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 18:37:24.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 18:37:24.905 Thread 'MainThread': missing ScriptRunContext! This w